In [1]:
import pandas as pd
import numpy as np
import joblib
import os

class FlowScoreAgent:
    def __init__(self, model_path):
        """
        에이전트 초기화: 학습된 모델을 로드합니다.
        """
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"모델 파일이 없습니다: {model_path}")
        
        self.model = joblib.load(model_path)
        self.features = [
            'Current_Ratio', 'Debt_Ratio', 'Late_Pay_Ratio', 'Avg_Delay_Days', 'Biz_Score', 'CEO_Score',
            'Sales_Growth', 'OPM_Change', 'Rev_Per_Emp', 'Tx_Volatility', 'Weekend_Tx_Ratio', 
            'Avg_Tx_Hour', 'Emp_Momentum'
        ]
        print("🤖 [FlowScore Agent] 시스템 가동 준비 완료 (Model v5.0 Loaded)")

    def _preprocess(self, raw_data):
        """
        원천 데이터를 모델이 이해하는 피처로 변환 (Feature Engineering 자동화)
        """
        data = raw_data.copy()
        
        # 1. 자동 계산 로직 (사용자가 원천 값만 넣었을 경우를 대비)
        # 매출 성장률 자동 계산
        if 'Sales_Curr' in data and 'Sales_Prev' in data:
            if data['Sales_Prev'] > 0:
                data['Sales_Growth'] = (data['Sales_Curr'] - data['Sales_Prev']) / data['Sales_Prev']
            else:
                data['Sales_Growth'] = 0.0
        
        # 유동비율 자동 계산
        if 'Current_Assets' in data and 'Current_Liabilities' in data:
            data['Current_Ratio'] = data['Current_Assets'] / data['Current_Liabilities'] if data['Current_Liabilities'] > 0 else 0

        # 누락된 필드 기본값 처리 (Safety)
        for feat in self.features:
            if feat not in data:
                data[feat] = 0 # 혹은 평균값 대체
        
        # 모델 입력 순서에 맞춰 정렬
        return pd.DataFrame([data])[self.features]

    def _explain_decision(self, data, prob):
        """
        판단 근거 설명 (Explainable AI)
        """
        reasons = []
        row = data.iloc[0]
        
        # 주요 거절 사유 로직 (모델의 중요도 기반)
        if row['Late_Pay_Ratio'] > 0.3: reasons.append(f"오후결제과다({row['Late_Pay_Ratio']*100:.0f}%)")
        if row['Debt_Ratio'] > 300: reasons.append(f"부채비율위험({row['Debt_Ratio']:.0f}%)")
        if row['Tx_Volatility'] > 0.4: reasons.append("자금흐름불안정")
        if row['Sales_Growth'] < -0.05: reasons.append("역성장추세")
        if row['CEO_Score'] < 700: reasons.append("대표자신용유의")
        if row['Avg_Tx_Hour'] >= 17: reasons.append("늦은결제습관")
        
        if prob < 0.5 and not reasons:
            reasons.append("종합평가미달")
        elif prob >= 0.5 and not reasons:
            reasons.append("우량지표다수")
            
        return ", ".join(reasons)

    def evaluate(self, company_name, raw_input):
        """
        신규 기업 평가 실행
        """
        # 1. 전처리
        input_df = self._preprocess(raw_input)
        
        # 2. 모델 예측
        prob = self.model.predict_proba(input_df)[0][1] # 승인 확률
        
        # 3. 결과 해석
        status = "✅ 승인 (Approve)" if prob >= 0.5 else "🚫 거절 (Reject)"
        risk_score = (1 - prob) * 100
        reason = self._explain_decision(input_df, prob)
        
        # 등급 부여
        if risk_score >= 80: grade = 'D (위험)'
        elif risk_score >= 50: grade = 'C (주의)'
        elif risk_score >= 20: grade = 'B (관찰)'
        else: grade = 'A (정상)'

        # 4. 리포트 출력
        print(f"\n📄 [심사 결과 리포트] - {company_name}")
        print("-" * 50)
        print(f"결  과 : {status}")
        print(f"점  수 : {risk_score:.1f}점 (등급: {grade})")
        print(f"확  률 : 승인 가능성 {prob*100:.1f}%")
        print(f"소  견 : {reason}")
        print("-" * 50)
        
        return {'Name': company_name, 'Status': status, 'Score': risk_score, 'Reason': reason}

# ==============================================================================
# 🚀 에이전트 가동 테스트 (Scenario Test)
# ==============================================================================

# 1. 에이전트 생성
agent = FlowScoreAgent('FlowScore_5.0_Final.pkl')

# 시나리오 1: (주)스타트업A - 매출은 급성장 중이나 자금 관리가 엉망인 케이스
new_company_A = {
    # 기초 재무
    'Current_Assets': 120, 'Current_Liabilities': 100, # 유동비율 1.2 (정상)
    'Debt_Ratio': 150, 
    'Sales_Curr': 500, 'Sales_Prev': 300, # 매출성장률 +66% (아주 좋음)
    
    # 활동성 (함정)
    'Late_Pay_Ratio': 0.6,    # 오후 결제 60% (위험!)
    'Tx_Volatility': 0.8,     # 변동성 큼 (위험!)
    'Avg_Tx_Hour': 18,        # 오후 6시 결제 (위험!)
    'Weekend_Tx_Ratio': 0.1,  # 주말 거래 존재
    
    # 기타
    'Biz_Score': 75, 'CEO_Score': 800, 'Avg_Delay_Days': 5, 'Rev_Per_Emp': 200, 'OPM_Change': 0.05, 'Emp_Momentum': 0.1
}

# 시나리오 2: (주)중견기업B - 성장은 멈췄지만 관리가 완벽한 케이스
new_company_B = {
    'Current_Ratio': 2.5, 'Debt_Ratio': 50,
    'Sales_Growth': 0.01, # 성장 정체
    'Late_Pay_Ratio': 0.0, 'Tx_Volatility': 0.1, 'Avg_Tx_Hour': 10, # 관리 완벽
    'Biz_Score': 90, 'CEO_Score': 950,
    'Avg_Delay_Days': 0, 'Rev_Per_Emp': 500, 'OPM_Change': 0.02, 'Emp_Momentum': 0.0, 'Weekend_Tx_Ratio': 0.0
}

# 심사 요청
print("\n>>> 🏢 신규 심사 요청 2건이 접수되었습니다.")
result_A = agent.evaluate("(주)스타트업A", new_company_A)
result_B = agent.evaluate("(주)중견기업B", new_company_B)

🤖 [FlowScore Agent] 시스템 가동 준비 완료 (Model v5.0 Loaded)

>>> 🏢 신규 심사 요청 2건이 접수되었습니다.

📄 [심사 결과 리포트] - (주)스타트업A
--------------------------------------------------
결  과 : 🚫 거절 (Reject)
점  수 : 83.5점 (등급: D (위험))
확  률 : 승인 가능성 16.5%
소  견 : 오후결제과다(60%), 자금흐름불안정, 늦은결제습관
--------------------------------------------------

📄 [심사 결과 리포트] - (주)중견기업B
--------------------------------------------------
결  과 : 🚫 거절 (Reject)
점  수 : 59.5점 (등급: C (주의))
확  률 : 승인 가능성 40.5%
소  견 : 종합평가미달
--------------------------------------------------
